I guess that for doc2vec it is not wise to use text pairs, because it will better predict the author of the text we fit into it rather than just 1/0 value of whether the text is written by the same author or not.
On the other hand, doc2vec is a way to vectorize documents, and I think we can try to use it instead of tf-idf.

Also found an example f how to use doc2vec with log regression: https://towardsdatascience.com/multi-class-text-classification-with-doc2vec-logistic-regression-9da9947b43f4

In [1]:
# conda install -c anaconda gensim

import numpy as np
import json
import re
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.ensemble import RandomForestClassifier

In [2]:
# I like them :)

def rand_emot():
    e = ["(o_o)",":-)",":P",":D","x)","ᓚᘏᗢ","╯°□°）╯︵ ┻━┻",":)",
         "*<:-)","^_^","(⌐■_■)","¯\_(ツ)_/¯", "(T_T)",":o","OwO",
        "( ͡❛ ͜ʖ ͡❛)","(̶◉͛‿◉̶)","( ≖.≖)","(ㆆ_ㆆ)","ʕ•́ᴥ•̀ʔっ","( ◡́.◡̀)","(^◡^ )"]
    return random.choice(e)

In [3]:
# just test the model on the tiny sample

"""
blablabla= [
   'когда всё это закончится',
   'плачет киска в коридоре у неё большое горе',
   'когда теряет равновесие твоё сознание усталое',
   'трактор велосипед дом стена бумажка кресло',
   'почему всё не так вроде всё как всегда'
]
 
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(some_texts)]
 
model = Doc2Vec(documents, vector_size = 5, workers = 4, epochs = 3)
"""

"\nblablabla= [\n   'когда всё это закончится',\n   'плачет киска в коридоре у неё большое горе',\n   'когда теряет равновесие твоё сознание усталое',\n   'трактор велосипед дом стена бумажка кресло',\n   'почему всё не так вроде всё как всегда'\n]\n \ndocuments = [TaggedDocument(doc, [i]) for i, doc in enumerate(some_texts)]\n \nmodel = Doc2Vec(documents, vector_size = 5, workers = 4, epochs = 3)\n"

In [4]:
def rand_emot():
    e = ["(o_o)",":-)",":P",":D","x)","ᓚᘏᗢ","╯°□°）╯︵ ┻━┻",":)",
         "*<:-)","^_^","(⌐■_■)","¯\_(ツ)_/¯", "(T_T)",":o","OwO",
        "( ͡❛ ͜ʖ ͡❛)","(̶◉͛‿◉̶)","( ≖.≖)","(ㆆ_ㆆ)","ʕ•́ᴥ•̀ʔっ","( ◡́.◡̀)","(^◡^ )"]
    return random.choice(e)

def load_files():
    text_pairs = [] #Would be nice to have as np.array
    labels = []
    fandom = []
    
    pair_id = []
    true_id = []
    author = []
    
    #Load truth JSON
    for line in open('/Users/olgaiarygina/Downloads/LP2/data/modified/train_truth.jsonl'):
        d = json.loads(line.strip())
        labels.append(int(d['same']))
        true_id.append(d['id'])
        author.append(d['authors'])
        

    #Load actual fanfic.
    print("loading fanfic...",rand_emot())
    for line in tqdm(open('/Users/olgaiarygina/Downloads/LP2/data/modified/train_pair.jsonl')):
        d = json.loads(line.strip())
        text_pairs.append(d['pair'])
        fandom.append(d['fandoms'])
        pair_id.append(d['id'])


    print("done loading",rand_emot())
    
    return text_pairs, labels, fandom, pair_id, true_id, author

In [5]:
text_pairs, labels, fandom, pair_id, true_id, author = load_files()

161it [00:00, 1604.63it/s]

loading fanfic... ╯°□°）╯︵ ┻━┻


1578it [00:00, 2543.23it/s]

done loading :-)


In [6]:
flat_list = [item for sublist in text_pairs for item in sublist]
flat_author = [item for sublist in author for item in sublist]

documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(flat_list)]

In [7]:
# test on a very small dataset

"""
little_doc_test = documents[:15]
little_author_test = author[:15]
model = Doc2Vec(little_doc_test, vector_size = 5, workers = 4, epochs = 3) # epochs and vector_size are chosen intuitively

# fit Random Forest
clf = RandomForestClassifier()
clf.fit([model.infer_vector([x.words]) for x in little_doc_test], little_author_test)
  
res = clf.predict([model.infer_vector(['text', 'Harry', 'irrestible'])]) # don't ask me why these words
print(res) 
"""


"\nlittle_doc_test = documents[:15]\nlittle_author_test = author[:15]\nmodel = Doc2Vec(little_doc_test, vector_size = 5, workers = 4, epochs = 3) # epochs and vector_size are chosen intuitively\n\n# fit Random Forest\nclf = RandomForestClassifier()\nclf.fit([model.infer_vector([x.words]) for x in little_doc_test], little_author_test)\n  \nres = clf.predict([model.infer_vector(['text', 'Harry', 'irrestible'])]) # don't ask me why these words\nprint(res) \n"

In [8]:
#Takes a while!

#model.build_vocab(documents)
model = Doc2Vec(documents, vector_size = 100, workers = 4, epochs = 15) # epochs and vector_size are chosen intuitively following some instructions from stack

In [9]:
model

In [10]:
# нахождение наиболее похожего документа
# finding the most similar docs

"""
vector_to_search = model.infer_vector(["find", "similar", "text"])

# три наиболее похожих
# 3 most similar
similar_documents = model.docvecs.most_similar([vector_to_search], topn = 3)
for s in similar_documents:
    print(flat_list[s[0]])
    """

'\nvector_to_search = model.infer_vector(["find", "similar", "text"])\n\n# три наиболее похожих\n# 3 most similar\nsimilar_documents = model.docvecs.most_similar([vector_to_search], topn = 3)\nfor s in similar_documents:\n    print(flat_list[s[0]])\n    '

In [ ]:
# fit Random Forest
# for the whole data takes a while as well

clf = RandomForestClassifier()
clf.fit([model.infer_vector([x.words]) for x in documents], flat_author)
  
res = clf.predict([model.infer_vector(['text', 'Harry', 'irrestible'])]) # don't ask me why these words
print(res)

In [ ]:
labels[1]